In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### classes clean_dataset

In [5]:
clean_df = pd.read_csv('../bva/data/clean_dataset.csv')
clean_df.head()

,Unnamed: 0,video_path,frame,birdie_visible,birdie_x,birdie_y,birdie_hit,stroke
0,0,match12/rally_video/1_01_00.mp4,0,0,0,0,0,NaN
1,1,match12/rally_video/1_01_00.mp4,1,0,0,0,0,NaN
2,2,match12/rally_video/1_01_00.mp4,2,0,0,0,0,NaN
3,3,match12/rally_video/1_01_00.mp4,3,0,0,0,0,NaN
4,4,match12/rally_video/1_01_00.mp4,4,0,0,0,0,NaN


In [8]:
clean_df['stroke'].unique()

def set_att_def(x):
    if x in ['short_serve','drop','net_shot','drive','half_smash','full_smash','net_kill']:
        return 'att'
    if x in ['long_serve','lob','clear','short_def','lift','long_def']:
        return 'def'
    return 'no_hit'

clean_df['stroke_cat'] = clean_df.apply(lambda x: set_att_def(x['stroke']), axis=1)
clean_df['stroke_cat'].unique()




array(['no_hit', 'att', 'def'], dtype=object)

### arrange prediction

In [34]:
match9 = clean_df[clean_df['video_path'] == 'match9/rally_video/1_07_11.mp4']
match9[match9['birdie_hit'] == 1]

,Unnamed: 0,video_path,frame,birdie_visible,birdie_x,birdie_y,birdie_hit,stroke,stroke_cat
45465,45465,match9/rally_video/1_07_11.mp4,10,0,0,0,1,short_serve,att
45486,45486,match9/rally_video/1_07_11.mp4,31,1,827,430,1,net_shot,att
45507,45507,match9/rally_video/1_07_11.mp4,52,1,814,379,1,lob,def
45540,45540,match9/rally_video/1_07_11.mp4,85,1,488,426,1,clear,def
45568,45568,match9/rally_video/1_07_11.mp4,113,1,769,202,1,drop,att
45582,45582,match9/rally_video/1_07_11.mp4,127,1,793,369,1,net_shot,att
45609,45609,match9/rally_video/1_07_11.mp4,154,1,448,411,1,net_shot,att
45633,45633,match9/rally_video/1_07_11.mp4,178,1,455,388,1,net_shot,att
45655,45655,match9/rally_video/1_07_11.mp4,200,1,786,377,1,lob,def
45685,45685,match9/rally_video/1_07_11.mp4,230,1,449,291,1,half_smash,att


In [29]:
import pandas as pd

In [36]:
from bva.analyze_predicts import find_final_predict_from_hitnet

result = find_final_predict_from_hitnet('../bva/data/hitnet_predict_match9_1_07_11.csv', 0.85)
result[result['hit'] == 1]


,frame,hit
10,10,1
31,31,1
51,51,1
84,84,1
128,128,1
153,153,1
177,177,1
199,199,1
232,232,1


,frame,hit
10,10,1
31,31,1
51,51,1
84,84,1
128,128,1
153,153,1
177,177,1
199,199,1
231,231,1


In [124]:
dim1 = []
dim2 = []
for i in range(10):
    dim2 = []
    for j in range(5):
        dim3 = []
        for k in range(3):
            test = [1,2,3,4]
            dim3.append(test)
        dim2.append(dim3)
    dim1.append(dim2)
    
np.array(dim1).shape
    



(10, 5, 3, 4)

### Normalization x et y

In [72]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [108]:
from bva.preprocess import get_all_videos_sequences_by_window

video_details_path = '../bva/data/video_details.csv'
clean_dataset_path = '../bva/data/clean_dataset.csv'

df, X, y, test_dict = get_all_videos_sequences_by_window(video_details_path, clean_dataset_path, 6, 5)

In [118]:
test_dict.keys()

dict_keys(['match9/rally_video/1_02_03.mp4', 'match9/rally_video/1_04_05.mp4', 'match9/rally_video/1_05_06.mp4', 'match9/rally_video/1_07_10.mp4', 'match9/rally_video/1_07_11.mp4'])

In [116]:
#plt.hist(y)
#print(y[0:30])
print(X.shape)
print(y.shape)
len(np.unique(y))

(1431, 6, 11)
(1431,)


13

### Preprocess

In [60]:
from bva.preprocess import get_all_videos_sequences_by_window
df, X, y, test_dict = get_all_videos_sequences_by_window(5, 5)

### Preprocess file from tracknet

In [67]:
predict_path = '../raw_data/1_00_02_predict.csv'
video_details_path = '../raw_data/1_00_02_details.csv'

filename = video_path.split('/')[-1].replace('.csv','.mp4')
video_path = f'./input/{filename}'

video_details = pd.read_csv(video_details_path)
video_details['video_path'] = video_path

video_birdie_positions = pd.read_csv(predict_path)
video_birdie_positions['video_path'] = video_path

video_birdie_positions = video_birdie_positions.rename(
                    columns = {'Frame':'frame', 
                                'Visibility':'visibility', 
                                'X': 'x', 
                                'Y':'y', 
                                'Time': 'time'}) \
                        .reindex(columns = ['video_path', 'frame', 'visibility', 'x', 'y', 'time'])

video_birdie_positions = video_birdie_positions.merge(video_details, on='video_path')

video_birdie_positions.head()


,video_path,frame,visibility,x,y,time,Unnamed: 0,fps,width,height,...,ur_corner_x,ur_corner_y,br_corner_x,br_corner_y,bl_corner_x,bl_corner_y,left_net_x,left_net_y,right_net_x,right_net_y
0,./input/1_00_02_predict.mp4,0,1,678,378,00:00:00.0,0,29.97,1280,720,...,832,393,1076,713,187,713,353,352,909,354
1,./input/1_00_02_predict.mp4,1,1,678,380,00:00:00.033366700033366704,0,29.97,1280,720,...,832,393,1076,713,187,713,353,352,909,354
2,./input/1_00_02_predict.mp4,2,1,681,383,00:00:00.06673340006673341,0,29.97,1280,720,...,832,393,1076,713,187,713,353,352,909,354
3,./input/1_00_02_predict.mp4,3,1,683,386,00:00:00.1001001001001001,0,29.97,1280,720,...,832,393,1076,713,187,713,353,352,909,354
4,./input/1_00_02_predict.mp4,4,1,686,395,00:00:00.13346680013346682,0,29.97,1280,720,...,832,393,1076,713,187,713,353,352,909,354


In [70]:
to_insert = ['nan'] * 5
y = ['1','2','3','4','5']
y = np.insert(y, 0, to_insert)
y

array(['n', 'n', 'n', 'n', 'n', '1', '2', '3', '4', '5'], dtype='<U1')